In [ ]:
# SIH25 Backend & Frontend: Quick Colab Launcher

# This Colab notebook will help you and your teammates run both the backend (FastAPI) and frontend (React) for the SIH25 multilingual chatbot project with minimal setup.

# **What this notebook does:**
# - Installs all required Python and Node.js dependencies
# - Prompts for your ngrok authtoken and LLM API keys (Groq, Gemini)
# - Clones the latest code from GitHub
# - Starts the backend server (FastAPI) and exposes it via ngrok
# - Starts the frontend (Vite React app) and exposes it via ngrok
# - Prints public URLs for both backend and frontend

# ---

In [ ]:
# Install all dependencies and import libraries
# Upgrade Node.js to v18 (required for Vite/React)
!curl -fsSL https://deb.nodesource.com/setup_18.x | bash -
!apt-get install -y nodejs
!node -v
!npm -v
# Python, Node.js, and ngrok setup
!pip install fastapi uvicorn python-dotenv googletrans==4.0.0-rc1 sqlalchemy requests pyngrok --quiet

import os
from pyngrok import ngrok
import threading
import time
import requests
from dotenv import load_dotenv

In [ ]:
# Kill stray backend/frontend processes on ports 8000 and 5173
import os
for port in [8000, 5173]:
    try:
        output = os.popen(f"lsof -ti tcp:{port}").read()
        pids = [pid for pid in output.strip().split('\n') if pid]
        for pid in pids:
            print(f"Killing process {pid} on port {port}")
            os.system(f"kill -9 {pid}")
    except Exception as e:
        print(f"Error killing processes on port {port}: {e}")
print("Done. If you still see 'address already in use', restart the runtime.")

In [ ]:
# Ask for ngrok authtoken and LLM API keys
NGROK_AUTH_TOKEN = input('Enter your ngrok authtoken: ')
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

llm_choice = input('Which LLM do you want to use? (groq/gemini/both): ').strip().lower()

if llm_choice in ('groq', 'both'):
    GROQ_API_KEY = input('Enter your Groq API key: ')
    os.environ['GROQ_API_KEY'] = GROQ_API_KEY
if llm_choice in ('gemini', 'both'):
    GEMINI_API_KEY = input('Enter your Gemini API key: ')
    os.environ['GEMINI_API_KEY'] = GEMINI_API_KEY

In [ ]:
# Clone repo and install backend/frontend dependencies
!git clone https://github.com/Kukyos/NullChat.git
%cd NullChat/backend
!pip install -r ../backend/requirements.txt --quiet
%cd ../frontend
!npm install --legacy-peer-deps > /dev/null
%cd ../backend

In [ ]:
# Start FastAPI backend with ngrok
load_dotenv()
public_url_backend = ngrok.connect(8000, bind_tls=True).public_url
print(f"Public FastAPI endpoint: {public_url_backend}")

def run_backend():
    import uvicorn
    uvicorn.run("app.main:app", host="0.0.0.0", port=8000, reload=False)

backend_thread = threading.Thread(target=run_backend)
backend_thread.start()
time.sleep(5)

In [ ]:
# Start Vite React frontend with ngrok
%cd ../frontend
public_url_frontend = ngrok.connect(5173, bind_tls=True).public_url
print(f"Public Frontend (React) URL: {public_url_frontend}")

# Start Vite dev server in a thread
def run_frontend():
    os.system("npm run dev -- --host 0.0.0.0 --port 5173")

frontend_thread = threading.Thread(target=run_frontend)
frontend_thread.start()
%cd ../backend
# Wait a bit for frontend to start
time.sleep(10)

In [ ]:
# Test the /ask endpoint (backend)
url = f"{public_url_backend}/ask"
payload = {
    "query": "What are the hostel facilities?",
    "language": "en"
}

try:
    response = requests.post(url, json=payload)
    print("Backend /ask response:", response.json())
except Exception as e:
    print("Error calling backend API:", e)

print(f"\nShare this backend API URL with your teammates: {public_url_backend}")
print(f"Share this frontend URL with your teammates: {public_url_frontend}")

---

**Instructions:**
- Run each cell in order.
- When prompted, paste your ngrok authtoken and API keys (they are not stored in the notebook or repo).
- The notebook will print public URLs for both backend and frontend—share these with your teammates for testing.

---

**Note:** This notebook is for demo/testing only. For production, deploy on a secure server and never share your API keys or ngrok token.